In [1]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
import json
from langchain_core.tools import ToolException

from pydantic.v1 import BaseModel, Field


import json
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
import requests
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
sectors_api_key = os.getenv("SECTORS_API_KEY")

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)

# dulu

In [63]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_groq import ChatGroq
import json


@tool
def get_daily_transaction_data(stock: str, start_date:str, end_date:str ) -> str:
    """Get daily transaction data for stock, such as close price, volume, and market cap on certain date"""
    stock = stock.lower()
    url = f"https://api.sectors.app/api/data/daily/{stock}/?start={start_date}&end={end_date}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        print(response.status_code)
    return result

@tool
def get_top_companies_based_on_transaction_volume(start_date:str, end_date:str, top_n:str) -> str:
    """Get top companies based on transaction volume"""

    url = f"https://api.sectors.app/api/data/most-traded/?start={start_date}&end={end_date}&n_stocks={top_n}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        print(response.status_code)

    return result

@tool
def get_company_info(stock:str) -> str:
  """ Get company info, such as company name, listing board, industry, sub industry, sector, sub sector, market cap, market cap rank, address, employee num, listing date, website, phone, email, last close price, latest close date, daily close change"""
  stock = stock.lower()
  url = f"https://api.sectors.app/api/data/company/report/{stock}/?sections=overview"
  api_key = sectors_api_key

  headers = {"Authorization": api_key}
  response = requests.get(url, headers = headers)

  if response.status_code == 200:
    data = response.json()
    result = json.dumps(data)
  else:
    print(response.status_code)

  return result


# SIMPLE QUERY
simple_query = "Give me the top 3 companies based on transaction volume on 2 April 2024"

# INTERMEDIATE QUERY
intermediate_query = "Based on the closing prices of BBRI from April 1 to April 28, is the trend positive or negative? Give me the detailed reason"

# COMPLEX QUERY
complex_query = "What were the market cap of BBCA and BREN? Which one was higher? For a stock with a higher market capitalization, could you also provide me with the email, phone number, and website, as I am interested in investing there?"

queries = [simple_query, intermediate_query, complex_query]


prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data, get_top_companies_based_on_transaction_volume, get_company_info]

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor = AgentExecutor(agent=agent, tools=tools)

for query in queries:
  print("Question:", query)
  result = agent_executor.invoke({"input": query})
  print("Answer:",result['output'], "\n\n======================================================================================================================\n")

Question: Give me the top 3 companies based on transaction volume on 2 April 2024
Answer: The top 3 companies based on transaction volume on 2 April 2024 are:

1. GOTO.JK with a transaction volume of 2224757900
2. DOOH.JK with a transaction volume of 675247000
3. PNLF.JK with a transaction volume of 502243100 


Question: Based on the closing prices of BBRI from April 1 to April 28, is the trend positive or negative? Give me the detailed reason
Answer: Based on the closing prices of BBRI from April 1 to April 28, the trend is positive. 

The closing price started at 4730 on April 1 and ended at 4870 on April 28, which indicates an overall increase. Although there were some fluctuations in between, the overall trend is upward. 


Question: What were the market cap of BBCA and BREN? Which one was higher? For a stock with a higher market capitalization, could you also provide me with the email, phone number, and website, as I am interested in investing there?
Answer: Based on the market c

# Using tool decorator

In [76]:
@tool()
def get_daily_transaction_data(stock: str, start_date:str, end_date:str ) -> str:
    """Get daily transaction data for stock, such as close price, volume, and market cap on certain date"""
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    return result


prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "Based on the closing prices of BBRI from April 1 to April 28, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Based on the closing prices of BBRI from April 1 to April 28, is the trend positive or negative? Give me the detailed reason
Answer: Based on the closing prices of BBRI from April 1 to April 28, the trend is generally positive. 

Here's a detailed reason: 

The closing prices started at 4730 on April 1 and ended at 4870 on April 28, which indicates an overall increase of 140 points or approximately 2.96% over the period. 

Although there were some fluctuations in between, with the price dropping to as low as 4540 on April 14, the overall trend is upward. The price recovered and even reached a high of 4940 on April 26, before closing at 4870 on April 28.


It's working perfectly, but how if we input an invalid ticker format?

In [77]:
query = "Based on the closing prices of BBRII from April 1 to April 28, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Based on the closing prices of BBRII from April 1 to April 28, is the trend positive or negative? Give me the detailed reason


ValueError: No data for BBRII

It returns an error, since BBRII is not a valid ticker format. However, if we show this kind of error to user, it won't be a great options, and that's why we need Pydantic

In [80]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime

class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for, should only contain up to 4 characters")
    start_date: str = Field(description="Start date in yyyy-mm-dd format")
    end_date: str = Field(description="End date in yyyy-mm-dd format")
    
    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ValueError('Stock ticker should only contain up to 4 alphabetic characters')
        return v

    @validator('start_date', 'end_date')
    def validate_date_format(cls, v):
        try:
            datetime.strptime(v, '%Y-%m-%d')
        except ValueError:
            raise ValueError('Date must be in yyyy-mm-dd format')
        return v
    
@tool(args_schema=SearchInput)
def get_daily_transaction_data(stock: str, start_date:str, end_date:str ) -> str:
    """Get daily transaction data for stock, such as close price, volume, and market cap on certain date"""
    stock = stock.lower()
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError("No available data")
    
    return result

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data ]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "Based on the closing prices of BBRII from April 1 to April 28, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Based on the closing prices of BBRII from April 1 to April 28, is the trend positive or negative? Give me the detailed reason


ValidationError: 1 validation error for SearchInput
stock
  Stock ticker should only contain up to 4 alphabetic characters (type=value_error)

Great, now we have ValidationError. But, this way, ini masih gabagus, makae kita butuh StructuredTool

# Structured Tools

https://blog.langchain.dev/structured-tools/

In [109]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException

class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for, should only contain up to 4 characters")
    start_date: str = Field(description="Start date in yyyy-mm-dd format")
    end_date: str = Field(description="End date in yyyy-mm-dd format")
    
    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ToolException('Stock ticker should only contain up to 4 alphabetic characters')
        return v

    @validator('start_date', 'end_date')
    def validate_date_format(cls, v):
        try:
            datetime.strptime(v, '%Y-%m-%d')
        except ValueError:
            raise ToolException('Date must be in yyyy-mm-dd format')
        return v
    
def get_daily_transaction_data(stock: str, start_date:str, end_date:str ) -> str:
    """Get daily transaction data for stock, such as close price, volume, and market cap on certain date"""

    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data available for {stock}")
    return result

get_daily_transaction_data_tool = StructuredTool.from_function(
    func=get_daily_transaction_data,
    args_schema= SearchInput,
    handle_tool_error=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


In [190]:
query = "Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason


> Entering new AgentExecutor chain...

Invoking: `get_daily_transaction_data` with `{'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRI'}`


[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.JK", "date": "2024-04-02", "close": 5675, "volume": 412180900, "market_cap": 855738880622592}, {"symbol": "BBRI.JK", "date": "2024-04-03", "close": 5625, "volume": 333518700, "market_cap": 848199333969920}, {"symbol": "BBRI.JK", "date": "2024-04-04", "close": 5700, "volume": 131936100, "market_cap": 867146649305088}, {"symbol": "BBRI.JK", "date": "2024-04-05", "close": 5650, "volume": 178817800, "market_cap": 851969107296256}, {"symbol": "BBRI.JK", "date": "2024-04-16", "close": 5350, "volume": 600717500, "market_cap": 806731827380224}, {"symbol": "BBRI.JK",

'Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nThe highest closing price was 5925 on April 1, 2024, and the lowest closing price was 4830 on April 26, 2024. The closing prices generally decreased over the period, with some fluctuations. The overall direction is downward, indicating a negative trend.'

In [171]:
query = "Based on the closing prices of BBRII from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Based on the closing prices of BBRII from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason


> Entering new AgentExecutor chain...

Invoking: `get_daily_transaction_data` with `{'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRII'}`


Stock ticker should only contain up to 4 alphabetic charactersI apologize for the mistake. It seems that the stock ticker "BBRII" has more than 4 characters. I'll make sure to provide a correct input.

Can you please provide the correct stock ticker with up to 4 alphabetic characters? I'll be happy to assist you further.

> Finished chain.
Answer: I apologize for the mistake. It seems that the stock ticker "BBRII" has more than 4 characters. I'll make sure to provide a correct input.

Can you please provide the correct stock ticker with up to 4 alphabetic characters? I'll be happy to assist you further.


# Adding memory

Make sure the langchain version is 0.0.354, documentation: https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [211]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("placeholder", "{history}"), # add this or could be: MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [212]:
store = {} # This dictionary acts like a dummy database that save the message history
print("History:", store)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_executor_w_memory = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

History: {}


In [213]:
agent_executor_w_memory.invoke(
    {"input": "Hi, I'm Wilsen"},
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...
Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?

> Finished chain.


{'input': "Hi, I'm Wilsen",
 'history': [],
 'output': "Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"}

In [214]:
agent_executor_w_memory.invoke(
    {"input": "Who am I"},
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...
You're Wilsen! We just introduced ourselves a moment ago. Is there something specific you'd like to know or talk about, Wilsen?

> Finished chain.


{'input': 'Who am I',
 'history': [HumanMessage(content="Hi, I'm Wilsen"),
  AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?")],
 'output': "You're Wilsen! We just introduced ourselves a moment ago. Is there something specific you'd like to know or talk about, Wilsen?"}

In [215]:
agent_executor_w_memory.invoke(
    {"input": query},
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...

Invoking: `get_daily_transaction_data` with `{'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRI'}`


[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.JK", "date": "2024-04-02", "close": 5675, "volume": 412180900, "market_cap": 855738880622592}, {"symbol": "BBRI.JK", "date": "2024-04-03", "close": 5625, "volume": 333518700, "market_cap": 848199333969920}, {"symbol": "BBRI.JK", "date": "2024-04-04", "close": 5700, "volume": 131936100, "market_cap": 867146649305088}, {"symbol": "BBRI.JK", "date": "2024-04-05", "close": 5650, "volume": 178817800, "market_cap": 851969107296256}, {"symbol": "BBRI.JK", "date": "2024-04-16", "close": 5350, "volume": 600717500, "market_cap": 806731827380224}, {"symbol": "BBRI.JK", "date": "2024-04-17", "close": 5300, "volume": 244002000, "market_cap": 799192280727552}, {"symbol": "BBRI.JK", "date": "2024-04-18", "close": 5

{'input': 'Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason',
 'history': [HumanMessage(content="Hi, I'm Wilsen"),
  AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"),
  HumanMessage(content='Who am I'),
  AIMessage(content="You're Wilsen! We just introduced ourselves a moment ago. Is there something specific you'd like to know or talk about, Wilsen?")],
 'output': "Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nHere's a detailed reason:\n\nThe highest closing price was 5925 on April 1, 2024, and the lowest closing price was 4830 on April 26, 2024. Throughout the period, the closing prices generally decreased, with some fluctuations. The overall direction is downward, indicating a negative trend.\n\nTo give you a better idea, here's a brief summary 

In [216]:
agent_executor_w_memory.invoke(
    {"input": "how about based on market cap? is it also negative?"},
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...

Invoking: `get_daily_transaction_data` with `{'end_date': '2024-04-28', 'start_date': '2024-04-01', 'stock': 'BBRI'}`


[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.JK", "date": "2024-04-02", "close": 5675, "volume": 412180900, "market_cap": 855738880622592}, {"symbol": "BBRI.JK", "date": "2024-04-03", "close": 5625, "volume": 333518700, "market_cap": 848199333969920}, {"symbol": "BBRI.JK", "date": "2024-04-04", "close": 5700, "volume": 131936100, "market_cap": 867146649305088}, {"symbol": "BBRI.JK", "date": "2024-04-05", "close": 5650, "volume": 178817800, "market_cap": 851969107296256}, {"symbol": "BBRI.JK", "date": "2024-04-16", "close": 5350, "volume": 600717500, "market_cap": 806731827380224}, {"symbol": "BBRI.JK", "date": "2024-04-17", "close": 5300, "volume": 244002000, "market_cap": 799192280727552}, {"symbol": "BBRI.JK", "date": "2024-04-18", "close": 5

{'input': 'how about based on market cap? is it also negative?',
 'history': [HumanMessage(content="Hi, I'm Wilsen"),
  AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"),
  HumanMessage(content='Who am I'),
  AIMessage(content="You're Wilsen! We just introduced ourselves a moment ago. Is there something specific you'd like to know or talk about, Wilsen?"),
  HumanMessage(content='Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason'),
  AIMessage(content="Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nHere's a detailed reason:\n\nThe highest closing price was 5925 on April 1, 2024, and the lowest closing price was 4830 on April 26, 2024. Throughout the period, the closing prices generally decreased, with some fluctuations. The overall direction is downw

In [265]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi, I'm Wilsen"), AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"), HumanMessage(content="Hi, I'm Wilsen"), AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"), HumanMessage(content="Hi, I'm Wilsen"), AIMessage(content="Hi Wilsen! How's your day going so far?"), HumanMessage(content="Hi, I'm Wilsen"), AIMessage(content='Hi Wilsen! What brings you here today?'), HumanMessage(content='Who am I?'), AIMessage(content="You're Wilsen!"), HumanMessage(content='Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason'), AIMessage(content='Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nThe highest closing pric

# Add agent scratchpad / intermediate step

In [266]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("placeholder", "{history}"), # add this or could be: MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True, return_intermediate_steps=True)

In [250]:
from langchain.agents.format_scratchpad import format_to_openai_functions
agent_scratchpad = format_to_openai_functions(result['intermediate_steps'])
len(agent_scratchpad)
agent_scratchpad

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_d7d1', 'function': {'arguments': '{"start_date":"2024-04-01","end_date":"2024-04-28","stock":"BBRI"}', 'name': 'get_daily_transaction_data'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d1c0ada3-a21f-4fec-acd3-d5b0858d6d5c', tool_calls=[{'name': 'get_daily_transaction_data', 'args': {'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRI'}, 'id': 'call_d7d1'}], tool_call_chunks=[{'name': 'get_daily_transaction_data', 'args': '{"start_date":"2024-04-01","end_date":"2024-04-28","stock":"BBRI"}', 'id': 'call_d7d1', 'index': None}]),
 FunctionMessage(content='[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.JK", "date": "2024-04-02", "close": 5675, "volume": 412180900, "market_cap": 855738880622592}, {"symbol": "BBRI.JK", "date": "2024-04-03", "close": 5625, "volume":

In [274]:
store = {} # This dictionary acts like a dummy database that save the message history
print("History:", store)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_executor_w_memory = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

History: {}


In [275]:
result = agent_executor_w_memory.invoke(
    {"input": "Hi, I'm Wilsen"},
     config={"configurable": {"session_id": "abc123"}},
)

history = get_session_history("abc123")
intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])



> Entering new AgentExecutor chain...
Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?

> Finished chain.


In [276]:
result = agent_executor_w_memory.invoke(
    {"input": "Who am I?" },
    config={"configurable": {"session_id": "abc123"}},
)

intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])



> Entering new AgentExecutor chain...
You're Wilsen!

> Finished chain.


In [278]:
result = agent_executor_w_memory.invoke(
    {"input": query},
    config={"configurable": {"session_id": "abc123"}},
)

intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])



> Entering new AgentExecutor chain...

Invoking: `get_daily_transaction_data` with `{'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRI'}`


[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.JK", "date": "2024-04-02", "close": 5675, "volume": 412180900, "market_cap": 855738880622592}, {"symbol": "BBRI.JK", "date": "2024-04-03", "close": 5625, "volume": 333518700, "market_cap": 848199333969920}, {"symbol": "BBRI.JK", "date": "2024-04-04", "close": 5700, "volume": 131936100, "market_cap": 867146649305088}, {"symbol": "BBRI.JK", "date": "2024-04-05", "close": 5650, "volume": 178817800, "market_cap": 851969107296256}, {"symbol": "BBRI.JK", "date": "2024-04-16", "close": 5350, "volume": 600717500, "market_cap": 806731827380224}, {"symbol": "BBRI.JK", "date": "2024-04-17", "close": 5300, "volume": 244002000, "market_cap": 799192280727552}, {"symbol": "BBRI.JK", "date": "2024-04-18", "close": 5

In [279]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi, I'm Wilsen"), AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"), HumanMessage(content='Who am I?'), AIMessage(content="You're Wilsen!"), HumanMessage(content='Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason'), AIMessage(content='Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nThe highest closing price was 5925 on April 1, 2024, and the lowest closing price was 4830 on April 26, 2024. Throughout the period, the closing prices generally decreased, with some fluctuations. The overall direction is downward, indicating a negative trend.'), FunctionMessage(content='[{"symbol": "BBRI.JK", "date": "2024-04-01", "close": 5925, "volume": 217207800, "market_cap": 893436680994816}, {"symbol": "BBRI.

In [280]:
result

{'input': 'Based on the closing prices of BBRI from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason',
 'history': [HumanMessage(content="Hi, I'm Wilsen"),
  AIMessage(content="Hi Wilsen! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"),
  HumanMessage(content='Who am I?'),
  AIMessage(content="You're Wilsen!")],
 'output': 'Based on the closing prices of BBRI from April 1, 2024, to April 28, 2024, the trend is negative.\n\nThe highest closing price was 5925 on April 1, 2024, and the lowest closing price was 4830 on April 26, 2024. Throughout the period, the closing prices generally decreased, with some fluctuations. The overall direction is downward, indicating a negative trend.',
 'intermediate_steps': [(ToolAgentAction(tool='get_daily_transaction_data', tool_input={'start_date': '2024-04-01', 'end_date': '2024-04-28', 'stock': 'BBRI'}, log="\nInvoking: `get_daily_tran

In [281]:
result = agent_executor_w_memory.invoke(
    {"input": "how about based on market cap? is it also negative?"},
    config={"configurable": {"session_id": "abc123"}},
)

intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])



> Entering new AgentExecutor chain...
Let me analyze the market capitalization data.

The highest market capitalization was 893,436,680,994,816 on April 1, 2024, and the lowest was 728,320,488,505,344 on April 26, 2024. Throughout the period, the market capitalization generally decreased, with some fluctuations. The overall direction is downward, indicating a negative trend.

So, based on the market capitalization data, the trend is also negative.

> Finished chain.


# Stream

In [ ]:
llm_stream = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key, streaming= True)
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_daily_transaction_data_tool]

agent = create_tool_calling_agent(llm_stream, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "Based on the closing prices of BBRII from April 1 2024 to April 28 2024, is the trend positive or negative? Give me the detailed reason"
print("Question:", query)
result = agent_executor.stream({"input": query})
list(agent_executor.stream({"input": query}))
# print("Answer:",result['output'])

# Async

In [ ]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(await calculator.ainvoke({"a": 2, "b": 3}))
print(calculator.invoke({"a": 2, "b": 3}))

In [ ]:
@tool
async def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


try:
    print(await multiply.ainvoke({"a": 2, "b": 3}))
except NotImplementedError:
    print("Raised not implemented error. You should not be doing this.")